# 모델 아키텍처

In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import time
from tqdm import tqdm

In [24]:
# 1. 모델 아키텍처
class NewsSimilarityModel(nn.Module):
    """
    뉴스 유사도 계산 모델
    
    구조:
    1. 뉴스 A, B → Emb 1 (일단은 KLUE-BERT) → Context Vector (768차원)
    2. CV + 외부피처 → FCL (각각 별도) → Linear → 최종 벡터
    3. 두 최종 벡터 간 Cosine Similarity 계산
    """
    def __init__(self, 
                 embedding_model_name='klue/bert-base', # 학습 데이터셋 구성할 때의 임베딩 모델이랑 동일하게 설정해야 할 수 있음
                 embedding_dim=768, # BERT 임베딩 차원
                 external_feature_dim=10, # 외부 피처 차원(일단 10개로 넣어 둠)
                 fcl_hidden_dim=512, # FCL의 히든 레이어 차원
                 linear_output_dim=256): # 최종 출력 벡터 차원
        super().__init__()
        
        # 사전 학습 임베딩 모델 로드
        self.embedding_model = AutoModel.from_pretrained(embedding_model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(embedding_model_name)
        
        # 프리징 - 임베딩 모델의 파라미터는 학습하지 않음
        for name, param in self.embedding_model.named_parameters():
            param.requires_grad = False # 임베딩 모델의 파라미터 고정

        # 텍스트 임베딩 차원과 외부 피처 차원을 합쳐서 입력 차원 계산
        input_dim = embedding_dim + external_feature_dim

        # 현재 뉴스(기준)에 대한 FCL과 선형 레이어
        self.fcl_current = nn.Sequential(
            nn.Linear(input_dim, fcl_hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(fcl_hidden_dim, fcl_hidden_dim)
        )

        # 과거 뉴스(비교 대상)에 대한 FCL과 선형 레이어
        self.fcl_past = nn.Sequential(
            nn.Linear(input_dim, fcl_hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(fcl_hidden_dim, fcl_hidden_dim)
        )
        
        # 최종 출력 벡터를 위한 선형 레이어
        self.linear_current = nn.Linear(fcl_hidden_dim, linear_output_dim)
        self.linear_past = nn.Linear(fcl_hidden_dim, linear_output_dim)

    def get_text_embedding(self, texts, device):
        """
        입력된 텍스트 리스트를 BERT 임베딩으로 변환
        → [CLS] 토큰의 출력만 사용
        """
        inputs = self.tokenizer(
            texts,
            return_tensors='pt',
            padding=True,
            truncation=True,
            max_length=512
        )
        inputs = {k: v.to(device) for k, v in inputs.items()}
        with torch.no_grad(): # 프리징된 모델이므로 파라미터는 업데이트하지 않음
            outputs = self.embedding_model(**inputs)
            return outputs.last_hidden_state[:, 0, :] # [CLS] 토큰 출력 (batch_size, 768)

    def forward(self, news_a, news_b, ext_a, ext_b, device):
        """
        기준 뉴스 A와 비교 뉴스 B, 외부 피처들을 받아 각각의 최종 벡터를 반환
        """
        # 뉴스 본문 임베딩
        emb_a = self.get_text_embedding(news_a, device)
        emb_b = self.get_text_embedding(news_b, device)

        # 외부 피처와 concat → [text_embedding + external_feature]
        combined_a = torch.cat([emb_a, ext_a.to(device)], dim=1)
        combined_b = torch.cat([emb_b, ext_b.to(device)], dim=1)

        # 각각 FCL → Linear 통과시켜서 최종 벡터 추출
        current_vec = self.linear_current(self.fcl_current(combined_a))
        past_vec = self.linear_past(self.fcl_past(combined_b))
        return current_vec, past_vec


# 2. 손실 함수
class SimilarityLoss(nn.Module):
    def __init__(self, margin=0.5):
        super().__init__()
        self.margin = margin

    def forward(self, vec_a, vec_b, labels):
        """
        벡터 A, B 간의 코사인 유사도를 기반으로 손실 계산
        - 유사 뉴스(label=1): 유사도는 1에 가까워야 하므로 (1 - sim)^2
        - 비유사 뉴스(label=0): sim이 margin보다 크면 페널티 부여
        """
        cosine_sim = F.cosine_similarity(vec_a, vec_b, dim=1)

        # 양성 샘플 손실: 1과 가까울수록 손실이 작음
        pos_loss = labels * torch.pow(1 - cosine_sim, 2)

        # 음성 샘플 손실: margin보다 크면 손실 발생
        neg_loss = (1 - labels) * torch.pow(torch.clamp(cosine_sim - self.margin, min=0), 2)

        # 전체 손실: 양성 샘플 손실 + 음성 샘플 손실
        return torch.mean(pos_loss + neg_loss), cosine_sim

# 데이터셋 및 학습 코드

In [28]:
# 3. 데이터셋 클래스
class NewsDataset(Dataset):
    def __init__(self, news_a, news_b, feat_a, feat_b, labels):
        # 뉴스 A와 B, 외부 피처, 레이블 저장
        self.news_a = news_a
        self.news_b = news_b
        self.feat_a = feat_a
        self.feat_b = feat_b
        self.labels = labels

    def __len__(self):
        return len(self.news_a) # 전체 샘플 개수 반환

    def __getitem__(self, idx):
        # 하나의 샘플을 반환(뉴스 A, B와 외부 피처, 레이블)
        return {
            'news_a': self.news_a[idx],
            'news_b': self.news_b[idx],
            'feat_a': torch.FloatTensor(self.feat_a[idx]),
            'feat_b': torch.FloatTensor(self.feat_b[idx]),
            'label': torch.FloatTensor([self.labels[idx]])
        }

def custom_collate(batch):
    # 배치 단위로 텐서를 묶는 함수
    return {
        'news_a': [item['news_a'] for item in batch],
        'news_b': [item['news_b'] for item in batch],
        'feat_a': torch.stack([item['feat_a'] for item in batch]),
        'feat_b': torch.stack([item['feat_b'] for item in batch]),
        'label': torch.stack([item['label'] for item in batch])
    }


# 4. 학습 함수 (기존 모델을 인자로 받음)
def train_model(model, dataloader, criterion, num_epochs=10, lr=1e-4):
    device = next(model.parameters()).device # 모델이 올라간 디바이스 사용
    optimizer = torch.optim.Adam(model.parameters(), lr=lr) # Adam 옵티마이저

    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        for batch in dataloader:
            optimizer.zero_grad()
            # 두 뉴스와 외부 피처를 모델에 입력하여 벡터 추출
            vec_a, vec_b = model(
                batch['news_a'],
                batch['news_b'],
                batch['feat_a'].to(device),
                batch['feat_b'].to(device),
                device
            )
            # 손실 계산 및 역전파
            loss, _ = criterion(vec_a, vec_b, batch['label'].squeeze().to(device))
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f'Epoch {epoch+1}: Loss={total_loss / len(dataloader):.4f}')
    return model


# 5. 유사 뉴스 탐색 함수
def find_similar_news(model, current_news, current_feat, past_news_list, past_feat_list, top_k=5, batch_size=32):
    device = next(model.parameters()).device
    model.eval()

    current_feat_tensor = torch.FloatTensor(current_feat).unsqueeze(0).to(device) # 현재 뉴스 외부변수 텐서화
    similarities = []

    # tqdm 적용: 전체 batch 수를 기준으로 진행률 표시
    for i in tqdm(range(0, len(past_news_list), batch_size), desc='비교 중'):
        batch_news = past_news_list[i:i+batch_size]
        batch_feat = past_feat_list[i:i+batch_size]
        feat_batch = torch.FloatTensor(batch_feat).to(device)

        with torch.no_grad():
            vec_curr, vec_past = model(
                [current_news]*len(batch_news),
                batch_news,
                current_feat_tensor.repeat(len(batch_news), 1),
                feat_batch,
                device
            )
            batch_sim = F.cosine_similarity(vec_curr, vec_past, dim=1)
            for j in range(len(batch_news)):
                similarities.append((i + j, batch_sim[j].item(), batch_news[j]))

    return sorted(similarities, key=lambda x: x[1], reverse=True)[:top_k]


# 6. 실행 예시
if __name__ == '__main__':
    start = time.time()
    
    df = pd.read_csv('/Users/han-yeeun/final/db/news_2023_2025_with_stock_impact.csv')

    # 텍스트 없는 뉴스 제거
    df = df.dropna(subset=['text_combined'])

    # 날짜 처리
    df['wdate'] = pd.to_datetime(df['wdate'])

    # 외부 변수 컬럼 정의 및 개수 저장
    ext_cols = ['D-3', 'D-2', 'D-1', 'D+1', 'D+2', 'D+3', 'D+7', 'D+14', 'D+30']
    ext_dim = len(ext_cols)

    # 기준 뉴스 설정(가장 최신 뉴스 하나)
    base_idx = 0
    base_news = df.iloc[base_idx]['text_combined']
    base_feat = df.iloc[base_idx][ext_cols].astype(float).values
    base_date = df.iloc[base_idx]['wdate']

    # 기준 뉴스보다 과거 뉴스만 비교 대상으로 설정
    compare_df = df[df['wdate'] < base_date].reset_index(drop=True)
    compare_df = compare_df.head(100)
    compare_news = compare_df['text_combined'].tolist()
    compare_feats = compare_df[ext_cols].astype(float).values.tolist()

    dummy_labels = [1] * len(compare_news)  # 손실 계산용 임시 라벨

    # 데이터셋 구성
    dataset = NewsDataset(
        news_a=[base_news] * len(compare_news),
        news_b=compare_news,
        feat_a=[base_feat] * len(compare_news),
        feat_b=compare_feats,
        labels=dummy_labels
    )
    dataloader = DataLoader(dataset, batch_size=8, collate_fn=custom_collate)

    # 모델 및 학습
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = NewsSimilarityModel(external_feature_dim=ext_dim).to(device)
    criterion = SimilarityLoss()

    print(f'총 비교 대상 뉴스 개수: {len(compare_news)}')

    model = train_model(model, dataloader, criterion, num_epochs=1, lr=1e-4)

    results = find_similar_news(
        model=model,
        current_news=base_news,
        current_feat=base_feat,
        past_news_list=compare_news,
        past_feat_list=compare_feats,
        top_k=5
    )

    end = time.time()

    # 결과 출력
    print('\n기준 뉴스:\n', base_news[:200], '...\n')
    print('유사 뉴스 Top-5:')
    for i, sim, text in results:
        print(f'[유사도 {sim:.4f}] {text[:100]}...')

    print(f'\n비교 대상 {len(compare_news)}건 | 소요 시간: {end - start:.2f}초')

총 비교 대상 뉴스 개수: 100
Epoch 1: Loss=0.2546


비교 중:   0%|          | 0/4 [00:00<?, ?it/s]huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
비교 중: 100%|██████████| 4/4 [00:24<00:00,  6.10s/it]


기준 뉴스:
 [단독] 카카오페이, 2500만 회원 쓱·스마일페이 품나…간편결제 시장 빅3 경쟁 후끈 매각가 5000억 안팎 달할듯 결제시장 내 입지강화 포석 카카오페이 [사진 = 연합뉴스] 국내 대표 전자결제사업자인 카카오페이가 신세계이마트 산하 간편결제사업부 인수에 나섰다. 네이버페이·토스페이에 대항해 시장 점유율을 늘리려는 포석으로 해석된다. 23일 정보기술(IT) ...

유사 뉴스 Top-5:
[유사도 0.9408] [클릭 e종목]"SPC삼립, 안전사고에 투심회복 요원…투자의견·목표가↓" 투자의견 '매수'→'중립 하향 조정 목표주가 기존 대비 20.5% 하향 조정 IBK투자증권은 23일 SPC...
[유사도 0.9403] SPC삼립, 반복되는 안전사고…투심 회복 요원-IBK 투자의견 ‘중립’, 목표가는 5만 9000원으로 ‘하향’ [이데일리 박순엽 기자] IBK투자증권은 23일 SPC삼립(00561...
[유사도 0.9395] "SPC삼립, 반복되는 안전사고…투자의견·목표가↓"-IBK 사진=연합뉴스 IBK투자증권은 23일 SPC삼립에 대해 반복되는 안전사고로 악화된 투자심리의 회복이 요원해 보인다며 투자...
[유사도 0.9393] BGF, 주주환원 노력 시 주가 재평가 가능…목표가↑-흥국 [이데일리 신하연 기자] 흥국증권은 BGF(027410)에 대해 연간 실적 모멘텀이 부진한 가운데 기업 밸류업 프로그램에...
[유사도 0.9390] "삼성물산, 삼성바이오 분할로 낮은 밸류 매력 부각"-유진 송도 삼성바이오로직스 1&2캠퍼스 전경./사진=삼성바이오로직스 유진투자증권은 23일 삼성물산에 대해 "삼성바이오로직스 분...

비교 대상 100건 | 소요 시간: 85.38초
